In [ ]:
# UMAP script to reduce zircon geochemistry
#2024-10-03 MAAZ
#2025-06-26, EBL

# input CG2024 dataset and select parameters (if not already input)
# run UMAP
# test UMAP diagnostic plotting, local dimension, PCA RGB values for global structure
# create decision boundary - linear SVC
# ROC_AUC curve calculation
# Plot global geochemistry in UMAP


# Importing libraries and functions

In [ ]:
import os
import numpy as np
import pandas as pd
import umap
import umap.plot
# import scipy

import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc
import plotly.express as px #for plottly
import joblib #for saving UMAP model

from matplotlib import colors

#Machine learning libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV, ShuffleSplit, KFold
from sklearn.metrics import roc_curve, auc, balanced_accuracy_score, f1_score, roc_auc_score


#Advanced plotting libraries
from itertools import compress
from mpl_toolkits.mplot3d import Axes3D
from bokeh.plotting import output_notebook #for interactive plot
from matplotlib.colors import Normalize

#Script requirements
umap.plot.output_notebook() #resources= INLINE
%matplotlib inline
#%matplotlib widget #for 3d plot
sns.set_theme(style='white', context='notebook', rc={'figure.figsize':(14, 10)}) #anything smaller does not help with points

#Helper functions
def export_legend(legend, filepath2, expand=[-5,-5,5,5]):   

    fig  = legend.figure
    fig.canvas.draw()
    bbox  = legend.get_window_extent()
    bbox = bbox.from_extents(*(bbox.extents + np.array(expand)))
    bbox = bbox.transformed(fig.dpi_scale_trans.inverted())
    
    fig.savefig(filepath2, dpi="figure", bbox_inches=bbox)

def make_dir(destDir):
    image_dir = destDir
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)  

In [ ]:
#User input - change directories 
data_folder1 = # add here

file1 = "CG2024data_v4.csv"
file2 = "legend.png" #plot
file3 = "workable_table.xlsx" #for reproducibility
file4 = "standard_scaler.xml"
file5 = "umap_model.xml" 

trial_name = 'V4_zircongeochemistry' #IMPORTANT: change this name to avoid overwriting outputs


#Script begins

os.chdir(data_folder1)
print(data_folder1)

data_folder2 = os.path.join(os.path.dirname(data_folder1), 'outputs', trial_name)
make_dir(data_folder2)


filepath1 = os.path.join(data_folder1, file1)
filepath2 = os.path.join(data_folder2, file2)
filepath3 = os.path.join(data_folder2, file3)
filepath4 = os.path.join(data_folder2, file4)
filepath5 = os.path.join(data_folder2, file5)

#Load table (the column indexes below can be obtained from a data dictionary)
table1 = pd.read_csv(filepath1, low_memory=False)
table1.head()

range_imputed = list([0, 1, 2, 5, 6, 9, 10,11, 16, 18, 37,39,22,26,30,35,46,47,49,63])
range_imputed.extend(list([19,23,27,36,38,78,80,67,68,69,72,74,75,76,77]))


#Generating workable table
table2 = table1.iloc[:, range_imputed] 

#medicine 1
idx1 = table2.Temporality.isnull()
idx2 = table2["DepositBatholith"].isnull()
idx3 = table2["Sample"].isnull()
idx4 = table2["Continent"].isnull()
idx5 = table2["Composition"].isnull()
idx6 = table2["ID"].isnull()
idx7 = table2["District"].isnull()
idx8 = table2["U"].isnull()
idx9 = table2["Ti"].isnull()
idx10 = table2["La"].isnull()
idx11 = table2["Sm"].isnull()
idx12 = table2["Dy"].isnull()
idx13 = table2["Lu"].isnull()
idx14 = table2["Zr"].isnull()
idx15 = table2["Nb"].isnull()
idx16 = table2["Al"].isnull()
idx17 = table2["Zr_Age_Ma"].isnull()


table2.loc[idx1, "Temporality"] = 'Unknown'
table2.loc[idx2, "DepositBatholith"] = 'Unknown'
table2.loc[idx3, "Sample"] = 'Unknown'
table2.loc[idx4, "Continent"] = 'Unknown'
table2.loc[idx5, "Composition"] = 'Unknown'
table2.loc[idx6, "ID"] = 'Unknown'
table2.loc[idx7, "District"] = 'Unknown'
table2.loc[idx8, "U"] = 0
table2.loc[idx9, "Ti"] = 0
table2.loc[idx10, "La"] = 0
table2.loc[idx11, "Sm"] = 0
table2.loc[idx12, "Dy"] = 0
table2.loc[idx13, "Lu"] = 0
table2.loc[idx14, "Zr"] = 0
table2.loc[idx15, "Nb"] = 0
table2.loc[idx16, "Al"] = 0
table2.loc[idx17, "Zr_Age_Ma"] = 0

#medicine 2: do not drop string NAs
any_idx = table2.isna().any(axis=1)
table3 = table2.loc[np.invert(any_idx), :] # np.invert(any_idx)
table3.reset_index(inplace = True) #the index from the input table is preserved (for searching points)

data_start_idx = 21 #10
col_names = list(table3.columns)
a = table2.shape[0]
b = table3.shape[0]
c = col_names[data_start_idx:]

print(f"Table 2 has {a} rows and Table 3 without NA has {b} rows")
print(f"UMAP will use: {c}")
table3.head()
table3.tail()

# Loading and filtering data

## Printing populations for each category

In [ ]:
categoricals = ["ml_classes", "setID", "Dataset", "District", "DepositBatholith", "Temporality", "ID", "Sample", "Composition", "Continent", "Zr_Age_Ma"]
for i in range(len(categoricals)):
    temp = table3.loc[:, [categoricals[i]]].value_counts()
    print(temp)

#table3.loc[:, [categoricals[1]]].value_counts()

# Embedding space
### Run only once each time the notebook is opened. The stochastic process within UMAP wont repeat itself for all plots otherwise

Generating and saving model

In [ ]:
# run UMAP IF not using UMAP model and saved scaler from file, if these have been created already, run the cell below instead

try:
    del embedding
except:
    print('Processing for the first time')

components_output = 2 #default=2, dimensionality
neighbors_input = 20 #default=15, preservation of local (> singletons) vs global structure
min_dist_input = 0.003 #0.003, min. dist. of packing value (in low dimensional representation)

#data
data = table3.iloc[:, data_start_idx:].values
sc = StandardScaler()
scaled_data = sc.fit_transform(data) 
#print("Scaler mean: ", sc.mean_)
#print("Scaler scale: ", sc.scale_)

#umap object () for umap.plot
embedding = umap.UMAP(n_neighbors= neighbors_input,
                      min_dist= min_dist_input,
                      metric='correlation', 
                      n_components= components_output,
                     ).fit(scaled_data)  

#Saving data for reproducibility
table3.to_excel(filepath3, index=False) #processed table
joblib.dump(sc, filepath4) #scaler
joblib.dump(embedding, filepath5) #umap transform

In [ ]:
# run this IF using the saved scaler and UMAP transform from file, OR after running cell above

sc = joblib.load(filepath4) #scaler
embedding = joblib.load(filepath5) #umap transformation

embedding2 = embedding.embedding_

#Load table 
table3 = pd.read_excel(filepath3)
table3.tail()

In [ ]:
# UMAP diagnostic plotting, options can be pca, local_dim, vq

fig, ax = plt.subplots(figsize=(10, 6))
ax=umap.plot.diagnostic(embedding, diagnostic_type='local_dim',point_size=10, ax=ax)

## PCA plotting to test if global structure is retained

### first cell calculates CLR transformed PCA along with estimated ZrSiO content, second displays unscaled RGB values used to make UMAP coloured by PCA RGB points

In [ ]:
#script for PCA

emb = pd.DataFrame(embedding2)
# Apply centered log transformation to the dataset using a calculated residual ZrSiO value

#theoretical calc ZrO (1000000ppm- sum TE)
Sum_TE = ['P', 'Ce', 'Eu', 'Th', 'Hf', 'Calculated_La', 'Calculated_Pr', 'Imputed_Y',
          'Imputed_Nd', 'Imputed_Gd', 'Imputed_Er', 'Imputed_Yb', 'Imputed_Sm', 'Imputed_Dy', 'Imputed_Lu']


Dat_UMAP = pd.concat([table3, emb.iloc[:, [0, 1]]], axis=1)
Dat_UMAP['ZrSiO'] = 1_000_000 - table3[Sum_TE].sum(axis=1)


# CLR transformation
def centered_log_ratio_transform(X):
    """Centered log-ratio transformation for compositional data."""
    if np.any(X <= 0):
        raise ValueError("CLR transformation requires strictly positive values.")
    geometric_mean = np.exp(np.mean(np.log(X), axis=1)).to_numpy().reshape(-1, 1)
    return np.log(X / geometric_mean)

# Correlation circle plot
def plot_correlation_circle(loadings, title="Correlation Circle"):
    plt.figure(figsize=(7, 7))
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)
    circle = plt.Circle((0, 0), 1, color='gray', fill=False, linestyle='--')
    plt.gca().add_artist(circle)

    x_axis, y_axis = loadings.columns[0], loadings.columns[1]
    for i in range(loadings.shape[0]):
        plt.arrow(0, 0, loadings.iloc[i, 0], loadings.iloc[i, 1],
                  color='r', alpha=0.7, head_width=0.02)
        plt.text(loadings.iloc[i, 0]*1.1, loadings.iloc[i, 1]*1.1,
                 loadings.index[i], ha='center', va='center', fontsize=9)

    plt.title(title)
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.xlim(-1.1, 1.1)
    plt.ylim(-1.1, 1.1)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# PCA pipeline
def run_pca_pipeline(df, clr_columns, group_column='ml_classes', n_components=3):
    
    features = df[clr_columns]
    groups = df[group_column]

    clr_scaled = pd.DataFrame(centered_log_ratio_transform(features),
                            columns=features.columns, index=features.index)

    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(clr_scaled)

    pca_scores = pd.DataFrame(principal_components,
                              columns=[f'PC{i+1}' for i in range(n_components)],
                              index=df.index)
    pca_scores[group_column] = groups

    # Scree plot
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, n_components + 1),
             pca.explained_variance_ratio_, marker='o', linestyle='--')
    plt.title("Scree Plot")
    plt.xlabel("Principal Component")
    plt.ylabel("Explained Variance Ratio")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 2D Scatter Plot: PC1 vs PC2
    plt.figure(figsize=(7, 6))
    sns.scatterplot(data=pca_scores, x='PC1', y='PC2', hue=group_column, palette='Set2')
    plt.title('PC1 vs PC2 Scatter Plot')
    plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
    plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 3D Scatter Plot
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    for group in pca_scores[group_column].unique():
        group_data = pca_scores[pca_scores[group_column] == group]
        ax.scatter(group_data['PC1'], group_data['PC2'], group_data['PC3'], label=group)
    ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
    ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
    ax.set_zlabel(f"PC3 ({pca.explained_variance_ratio_[2]*100:.1f}%)")
    ax.set_title("PC1 vs PC2 vs PC3 (3D Scatter Plot)")
    ax.legend()
    plt.tight_layout()
    plt.show()

    # Loadings
    loadings = pd.DataFrame(pca.components_.T[:, :3],
                            columns=['PC1', 'PC2', 'PC3'],
                            index=features.columns)

    plot_correlation_circle(loadings[['PC1', 'PC2']], title="Correlation Circle: PC1 vs PC2")
    plot_correlation_circle(loadings[['PC1', 'PC3']], title="Correlation Circle: PC1 vs PC3")

    print("PCA Loadings:")
    print(loadings.round(3))

    return pca, pca_scores, loadings


if __name__ == "__main__":
    group_labels = table3['ml_classes']
    clr_columns = ['P', 'Ce', 'Eu', 'Th', 'Hf', 'Calculated_La', 'Calculated_Pr', 'Imputed_Y', 
                   'Imputed_Nd', 'Imputed_Gd', 'Imputed_Er', 'Imputed_Yb', 'Imputed_Sm', 'Imputed_Dy', 'Imputed_Lu', 'ZrSiO']
    Dat_UMAP['ml_classes'] = pd.Series(group_labels, index=Dat_UMAP.index[:len(group_labels)])  # ensure group_labels matches df index
    pca_model, pca_scores, pca_loadings = run_pca_pipeline(Dat_UMAP, clr_columns=clr_columns, group_column='ml_classes')


In [ ]:
#PCA to RGB to UMAP x, y


def pca_to_rgb(pca_scores):
    pcs = pca_scores[['PC1', 'PC2', 'PC3']].copy()
    pcs_norm = np.zeros_like(pcs.values)

    # PC 1-3 normalized to be from 0-1
    for i in range(pcs.shape[1]):
        norm = Normalize(vmin=pcs.iloc[:, i].min(), vmax=pcs.iloc[:, i].max())
        pcs_norm[:, i] = norm(pcs.iloc[:, i])

    # Convert to list of RGB tuples
    rgb_colors = [tuple(color) for color in pcs_norm]
    return np.array(rgb_colors)

#give colour for mapping
rgb_colors = pca_to_rgb(pca_scores)

#PCA biplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# PC1 vs PC2
axes[0].scatter(pca_scores['PC1'], pca_scores['PC2'],c=rgb_colors, s=20, alpha=0.9, edgecolor='k', linewidth=0.3)
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[0].set_title('PC1 vs PC2')

# PC1 vs PC3
axes[1].scatter(pca_scores['PC1'], pca_scores['PC3'],c=rgb_colors, s=20, alpha=0.9, edgecolor='k', linewidth=0.3)
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC3')
axes[1].set_title('PC1 vs PC3')

plt.tight_layout()
plt.show()


# UMAP plot using PCA colouring

col_x = Dat_UMAP.columns[-3]
col_y = Dat_UMAP.columns[-2]

plt.figure(figsize=(8, 6))
plt.scatter(Dat_UMAP[col_x], Dat_UMAP[col_y],c=rgb_colors, s=20, alpha=0.9, edgecolor='k', linewidth=0.3)
plt.xlabel(col_x)
plt.ylabel(col_y)
plt.title('UMAP Projection Colored by PCA RGB')
plt.tight_layout()
plt.show()

## Machine Learning Classification

In [ ]:
# enter variable below

variable_legend = "ml_classes" #e.g., Temporality, ml_classes

list_unique = table3[variable_legend].unique()
n_classes = len(list_unique)

#colourmap
mapping = {item:i for i, item in enumerate(list_unique)}
classif= table3[variable_legend].apply(lambda x: mapping[x]) #categorical array (same size as table3)
colourmap = sns.color_palette(palette= cc.glasbey_category10, n_colors = n_classes)
colours = [sns.color_palette(palette= colourmap)[x] for x in classif] #RGB triplets

colourmap_updated = colourmap #pre-allocating
for x in range(0, n_classes):
    
    idx = (classif == x)
    name = list_unique[x]

    colours_sub = colourmap[x]
    
    if name == 'Ore syn-mineral magmatism':
        colours_sub = colors.to_rgb('red')

    if name == 'Syn Mineral':
        colours_sub = colors.to_rgb('red')

    if name == 'Ore related magmatism':
        target_colour = (255, 208, 0)
        colours_sub = tuple(ti/255 for ti in target_colour)

    if name == 'Unknown':
        colours_sub = colors.to_rgb('lightgrey')

    if name == 'S Type Granite':
        colours_sub = colors.to_rgb('violet')

    colourmap_updated[x] = colours_sub

In [ ]:
# enter variable to be used as known labels for binary classification

variable_legend = "ml_classes" #Temporality

#medicine 3
if variable_legend == "Deposit/Batholith":
    variable_legend1 = "Deposit-Batholith"
else:
    variable_legend1 = variable_legend
    

list_unique = table3[variable_legend].unique()
n_classes = len(list_unique)

#colourmap
mapping = {item:i for i, item in enumerate(list_unique)}
classif= table3[variable_legend].apply(lambda x: mapping[x]) #categorical array (same size as table3)

#panda series
df = pd.DataFrame( dict(x=embedding2[:,0], y=embedding2[:,1], z= classif) ) 
X = df.iloc[:, 0:2].to_numpy()
Y = df.iloc[:, 2].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

In [ ]:
# binary decision plane using linear SVC, performance scores of binary model

clf = SVC(kernel="linear", C=0.025, random_state=42, probability= True) #C = 1 (regularisation)

clf.fit(X_train, y_train)
# score = clf.score(X_test, y_test) #mean accuracy 

 # Predict using the best model
best_model = clf #if cross-validation: .best_estimator_
pred = best_model.predict(X_test)
ac = balanced_accuracy_score(y_test, pred) # same as 'score'
f1_ = f1_score(y_test, pred) # F1 scores

# Receiver operating characteristic (ROC) area under the curve (AUC)
probs = best_model.predict_proba(X_test)
probs = probs[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probs)
auc_ = auc(fpr, tpr)

print("=== Nested K-Fold Cross-Validation Scores ===")
print("Mean balanced accuracy: "+ str(round(np.mean(ac), 3)))
print("Mean F1: "+ str(round(np.mean(f1_), 3)))
print("Mean roc_auc: "+ str(round(np.mean(auc_), 3)))


In [ ]:
# plotting ROC_AUC

probs = best_model.predict_proba(X_test)
probs = probs[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probs)
auc_ = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % auc_)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
# plot global projection of zircon geochemistry from CG2024

#Saving names
filepath3_new = filepath2.replace("legend.png", variable_legend1 + "_legend.png")
filepath4_new = filepath2.replace("legend.png", variable_legend1 + "_plot.png")
print(filepath4_new)

#Plot
markerSize = 6
fontSize = 18

fig = plt.figure(dpi= 400,figsize=(8,6)) #1200 , figsize=(10, 10)

for x in range(0, n_classes):

    
    idx = (classif == x)
    name = list_unique[x]
   
    # colours_sub = list(compress(colours, idx)) #required to index list
    colours_sub = np.asarray(colourmap_updated[x]).reshape(1,-1)        

    scatter = plt.scatter(embedding2[idx, 0], embedding2[idx, 1],
                              c=colours_sub, label = name,
                              s= 20, alpha= .7, edgecolors= 'none')

plt.xlabel("UMAP0")
plt.ylabel("UMAP1")

ax = plt.gca()
cm = plt.cm.RdBu

plt.gca().set_aspect('equal', 'datalim')

lgnd = plt.legend(ncol=1, fontsize= fontSize, loc='center right', bbox_to_anchor=(1.4, 0.5),
                  markerscale= 10, scatterpoints=1)
export_legend(lgnd, filepath2= filepath3_new)
lgnd.remove()
plt.show()

fig.savefig(filepath4_new, dpi="figure")
            